In [ ]:
import pandas as pd

file_path_1= '/home/mariam/Desktop/Campus/projet_datForGood/data/bilan-social-d-edf-sa-effectifs-et-repartition-par-age-statut-et-sexe.csv'
file_path_2= '/home/mariam/Desktop/Campus/projet_datForGood/data/bilan-social-d-edf-sa-salaries-en-situation-de-handicap.csv'
effectifs_df = pd.read_csv(file_path_1, delimiter=';')
handicap_df=pd.read_csv(file_path_2, delimiter=';')



In [2]:
effectifs_df

,Année,Perimètre juridique,Perimètre spatial,Spatial perimeter,Indicateur,Indicator,Type de contrat,Type of contract,Collège,Employee category,...,Nationalité,Nationality,Ancienneté,Seniority,Tranche d'âge,Age bracket,Valeur,Unité,Unit,Chapitre du bilan social
0,2023,EDF SA,France,France,Effectif,Workforce,Non statutaires CDI,Permanent non-staturory,Cadre,Managers,...,NaN,NaN,NaN,NaN,Moins de 25 ans,Less than 25,9.0,nombre,number,§1.1.6
1,2023,EDF SA,France,France,Effectif,Workforce,Non statutaires CDD,Fixed-term non-statutory,Cadre,Managers,...,NaN,NaN,NaN,NaN,Moins de 25 ans,Less than 25,19.0,nombre,number,§1.1.6
2,2023,EDF SA,France,France,Effectif,Workforce,Non statutaires CDD,Fixed-term non-statutory,Maîtrise,Supervisors,...,NaN,NaN,NaN,NaN,Moins de 25 ans,Less than 25,683.0,nombre,number,§1.1.6
3,2023,EDF SA,France,France,Effectif,Workforce,Non statutaires CDD,Fixed-term non-statutory,Exécution,Operatives,...,NaN,NaN,NaN,NaN,Moins de 25 ans,Less than 25,1216.0,nombre,number,§1.1.6
4,2023,EDF SA,France,France,Effectif,Workforce,Statutaires,Statutory,Cadre,Managers,...,NaN,NaN,NaN,NaN,De 25 à 35 ans,From 25 to 35,2046.0,nombre,number,§1.1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,2017,EDF SA,France,France,Embauches de salariés de moins de 25 ans,Hires of Employees Under 25 Years Old,Non statutaires CDD,Fixed-term non-statutory,Exécution,Operatives,...,NaN,NaN,NaN,NaN,NaN,NaN,367.0,nombre,number,§1.3.3
2318,2017,EDF SA,France,France,Démissions,Resignations,Statutaires,Statutory,Cadre,Managers,...,NaN,NaN,NaN,NaN,NaN,NaN,87.0,nombre,number,§1.4.2
2319,2017,EDF SA,France,France,Démissions,Resignations,Statutaires,Statutory,Maîtrise,Supervisors,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,nombre,number,§1.4.2
2320,2017,EDF SA,France,France,Démissions,Resignations,Non statutaires CDI,Permanent non-staturory,Maîtrise,Supervisors,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,nombre,number,§1.4.2


In [3]:
effectifs_df.rename(columns={
    'Genre': 'Sexe',
    'Type de contrat': 'Contrat',
    'Collège': 'Categorie',
    'Tranche d\'âge': 'Tranche_age',
    'Valeur': 'Valeur_effectifs',
    'Unité': 'Unite_effectifs'
}, inplace=True)

handicap_df.rename(columns={
    'Genre': 'Sexe',
    'Type de contrat': 'Contrat',
    'Collège': 'Categorie',
    'Valeur': 'Valeur_handicap',
    'Unité': 'Unite_handicap'
}, inplace=True)



In [ ]:
# Normaliser les valeurs de la colonne "Sexe"
effectifs_df['Sexe'] = effectifs_df['Sexe'].str.lower().map({'homme': 'Homme', 'femme': 'Femme'})
handicap_df['Sexe'] = handicap_df['Sexe'].str.lower().map({'homme': 'Homme', 'femme': 'Femme'})

In [ ]:
# Supprimer les doublons
effectifs_df.drop_duplicates(inplace=True)
handicap_df.drop_duplicates(inplace=True)

# Gérer les valeurs manquantes
effectifs_df.fillna({'Sexe': 'Non spécifié'}, inplace=True)
handicap_df.fillna({'Sexe': 'Non spécifié'}, inplace=True)

# 3. Ajout d'une colonne "Source"
effectifs_df['Source'] = 'Effectifs'
handicap_df['Source'] = 'Handicap'

In [ ]:
# Aligner les colonnes communes
columns_to_keep = ['Année', 'Sexe', 'Contrat', 'Categorie', 'Valeur_effectifs', 'Valeur_handicap', 'Source']
effectifs_df = effectifs_df[['Année', 'Sexe', 'Contrat', 'Categorie', 'Valeur_effectifs', 'Source']]
handicap_df = handicap_df[['Année', 'Sexe', 'Contrat', 'Categorie', 'Valeur_handicap', 'Source']]

In [ ]:
# Fusionner les deux jeux de données
merged_df = pd.merge(effectifs_df, handicap_df, on=['Année', 'Sexe', 'Contrat', 'Categorie', 'Source'], how='outer')
# Calculer le pourcentage d’hommes et de femmes
total_effectifs = merged_df.groupby(['Categorie'])['Valeur_effectifs'].transform('sum')
merged_df['Pourcentage_sexe'] = (merged_df['Valeur_effectifs'] / total_effectifs) * 100

# Calculer la part des personnes en situation de handicap dans les effectifs totaux
merged_df['Part_handicap'] = (merged_df['Valeur_handicap'] / merged_df['Valeur_effectifs']) * 100

# Calculer l'évolution annuelle (croissance/décroissance)
merged_df['Evolution_annuelle'] = merged_df.groupby(['Sexe', 'Contrat', 'Categorie'])['Valeur_effectifs'].pct_change() * 100


Préparation des données terminée. Fichier enregistré sous 'edf_bilan_social_preparé.csv'.


/tmp/ipykernel_11335/2566416972.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effectifs_df['Sexe'] = effectifs_df['Sexe'].str.lower().map({'homme': 'Homme', 'femme': 'Femme'})
/tmp/ipykernel_11335/2566416972.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effectifs_df.drop_duplicates(inplace=True)
/tmp/ipykernel_11335/2566416972.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [ ]:
merged_df.to_csv("edf_bilan_social_preparé.csv", index=False)

In [5]:
effectifs_df

,Année,Sexe,Contrat,Categorie,Valeur_effectifs,Source
0,2023,Homme,Non statutaires CDI,Cadre,9.0,Effectifs
1,2023,Femme,Non statutaires CDD,Cadre,19.0,Effectifs
2,2023,Femme,Non statutaires CDD,Maîtrise,683.0,Effectifs
3,2023,Homme,Non statutaires CDD,Exécution,1216.0,Effectifs
4,2023,Femme,Statutaires,Cadre,2046.0,Effectifs
...,...,...,...,...,...,...
2317,2017,Femme,Non statutaires CDD,Exécution,367.0,Effectifs
2318,2017,Homme,Statutaires,Cadre,87.0,Effectifs
2319,2017,Femme,Statutaires,Maîtrise,10.0,Effectifs
2320,2017,Homme,Non statutaires CDI,Maîtrise,0.0,Effectifs


In [6]:
handicap_df

,Année,Sexe,Contrat,Categorie,Valeur_handicap,Source
0,2023,Femme,Statutaires,Cadre,343.0,Handicap
1,2023,Homme,Non Statutaires CDI,Maîtrise,0.0,Handicap
2,2023,Homme,Non Statutaires CDD,Cadre,0.0,Handicap
3,2023,Non spécifié,Statutaires,Maîtrise,49.0,Handicap
4,2023,Non spécifié,Non Statutaires CDD,Cadre,0.0,Handicap
...,...,...,...,...,...,...
184,2017,Femme,Non Statutaires CDI,Exécution,1.0,Handicap
185,2017,Homme,Non Statutaires CDD,Cadre,0.0,Handicap
186,2017,Femme,Non Statutaires CDD,Cadre,0.0,Handicap
187,2017,Femme,Non Statutaires CDD,Maîtrise,10.0,Handicap


In [12]:
merged_df

,Année,Sexe,Contrat,Categorie,Valeur_effectifs,Source,Valeur_handicap,Pourcentage_sexe,Part_handicap,Evolution_annuelle
0,2023,Homme,Non statutaires CDI,Cadre,9.0,Effectifs,NaN,0.001073,NaN,NaN
1,2023,Homme,Non statutaires CDI,Cadre,137.0,Effectifs,NaN,0.016336,NaN,1422.222222
2,2023,Homme,Non statutaires CDI,Cadre,14.0,Effectifs,NaN,0.001669,NaN,-89.781022
3,2023,Homme,Non statutaires CDI,Cadre,8.0,Effectifs,NaN,0.000954,NaN,-42.857143
4,2023,Homme,Non statutaires CDI,Cadre,10.0,Effectifs,NaN,0.001192,NaN,25.000000
...,...,...,...,...,...,...,...,...,...,...
1985,2017,Femme,Non Statutaires CDI,Exécution,NaN,Handicap,1.0,NaN,NaN,NaN
1986,2017,Homme,Non Statutaires CDD,Cadre,NaN,Handicap,0.0,NaN,NaN,NaN
1987,2017,Femme,Non Statutaires CDD,Cadre,NaN,Handicap,0.0,NaN,NaN,NaN
1988,2017,Femme,Non Statutaires CDD,Maîtrise,NaN,Handicap,10.0,NaN,NaN,NaN


In [13]:
merged_df.to_csv("../data/EDF_effectifs_handi.csv", index=False)